# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

# Tool dimensions
TOOL_DIM = [0.32, 0.08]
TOOL_OFFSET = -0.04
ROBOT_Z_ANGLE = np.pi
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
ROBOT_Z_ANGLE = np.pi
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.set_workspace_boundary(-3, 7, -5, 5, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2-CLEARANCE), rpy=(0,0,0))

Current PC IP: 192.168.0.123
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10
connection command:
kmb0: False
indy1: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


### 0.2 Wait task start queue

## 1. Bed cleaning

### 1.1 Detect bed

#### 1.1.1 Move to bed-seek pose 

In [2]:
VIEW_POSE = np.deg2rad([  0., -28.,  85.,  -0.,  57., -180])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
gscene.show_pose(VIEW_POSE_EXT)

   Use a production WSGI server instead.
 * Debug mode: off


#### 1.1.2  detect bed and add to the scene

In [3]:
bed_center = (2,0,0)
bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

closet_left, closet_rightup, closet_rightdown = add_closet(
    gscene, closet_center=np.matmul(Rot_rpy(bed_rpy), (-0.75,-1,0))+bed_center, closet_rpy=bed_rpy, 
    COLOR_CLOSET_COL=(0,1,0,0.3))

### 1.2 Make bed cleaning plan

#### 1.2.1 get division-base pose combination data

In [4]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [5]:
wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
gcheck = GraspChecker(pscene)
ccheck = CachedCollisionCheck(gcheck, wp_task, wp_hdl, wayframer)

In [6]:
BED_OFFSET = 0.05
TABLE_DIMS = bed_mat.dims[1], bed_mat.dims[0], bed_mat.dims[2]
T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_DEPTH_OFF = T_brush_e[0, 3]
EE_HEIGHT = round(bed_mat.get_tf(HOME_DICT)[2,3] + bed_mat.dims[2]/2, 5) + T_brush_e[2, 3] + BED_OFFSET - INDY_BASE_OFFSET[2]

In [7]:
div_base_dict, Tsm_keys, surface_div_centers, sqdiv_size, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(bed_mat, brush_face, robot_config, 
                                          plane_val=EE_HEIGHT, tip_dir=None, TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck)

#### 1.2.2 select coering base poses and generate motions

In [ ]:
idx_bases, idc_divs, covered_all = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step)

In [9]:
# remove finished highlights
for i_b in idx_bases:
    hl_name = "sweep_fin_{}".format(i_b)
    if hl_name in gscene.NAME_DICT:
        gscene.remove(gscene.NAME_DICT[hl_name])

crob.home_pose = np.array([0,] * 12)
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    
mplan.motion_filters = [gcheck]
Qcur = crob.home_pose
for i_b, idc_div in zip(idx_bases, idc_divs):
    swp_centers = np.array(surface_div_centers)[idc_div]
    Tsm = T_xyzquat(Tsm_keys[i_b])
    set_base_sweep(pscene, floor_ws, Tsm, bed_mat, swp_centers, sqdiv_size, WP_DIMS, TOOL_DIM,  crob.home_dict)

    ppline.set_motion_planner(mplan)
    ppline.set_task_planner(tplan)
    initial_state = pscene.initialize_state(Qcur)

    ppline.search(initial_state, [(2, 1)], verbose=True,
                  timeout=0.3, timeout_loop=10, multiprocess=True,
                  add_homing=False, terminate_on_first=True, post_optimize=False)
    snode_schedule = tplan.get_best_schedule(at_home=False)
    snode_last = snode_schedule[-1]
    ref_state = snode_last.state.copy(pscene)
    ref_state.Q[crob.idx_dict[ROBOT_NAME]] = crob.home_pose[crob.idx_dict[ROBOT_NAME]]
    snode_schedule = snode_schedule+ppline.add_return_motion(snode_last, ref_state)
    ppline.play_schedule(snode_schedule)
    Qcur = snode_schedule[-1].state.Q
    Qcur[3:6] = 0
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_b), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)

Use 36/36 agents
try: 0 - (0, 0)->(0, 1)
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(1, 0)
try: 0 - (0, 0)->(1, 0)
try transition motion
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try transition motion
try: 0 - (0, 0)->(0, 1)
transition motion tried: True
try: 0 - (0, 0)->(1, 0)
try: 0 - (0, 0)->(0, 1)
try transition motion
result: 0 - (0, 0)->(0, 1) = success
branching: 0->1 (0.14/10.0 s, steps/err: 11(93.8990116119 ms)/0.00122546716747)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 1 - (0, 1)->(1, 1)
try: 1 - (0, 1)->(1, 1)
try transition motion
try transition motion
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 0 - (0, 0)->(1, 0)
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
try: 2 - (0, 1)->(1, 1)
try transition motion
try transition motion
branching

Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
try constrained motion
transition motion tried: False
try transition motion
result: 0 - (0, 0)->(1, 0) = fail
try transition motion
try: 0 - (0, 0)->(0, 1)
transition motion tried: False
Motion Plan Failure
Motion Plan Failure
try: 0 - (0, 0)->(0, 1)
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(1, 0)
result: 0 - (0, 0)->(1, 0) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
result: 0 - (0, 0)->(1, 0) = fail
transition motion tried: True
try: 1 - (0, 1)->(1, 1)
try transition motion
try transition motion
try transition motion
result: 2 - (0, 1)->(1, 1) = success
branching: 2->20 (0.49/10.0 s, steps/err: 41(126.598119736 ms)/0.000819214894835)
transition motion tried: False
try transition motion
Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
transition motion tried: False
transition motion tried: True
try: 20 - (1, 1)->(2, 1)
try: 0 - (0, 0)->(0, 1)
Motion Plan Failure
try: 0 - (0, 0)->(0, 1)
result: 7 -

try constrained motion
transition motion tried: True
result: 24 - (0, 1)->(1, 1) = success
try constrained motion
result: 0 - (0, 0)->(1, 0) = fail
result: 28 - (0, 1)->(1, 1) = success
branching: 24->39 (0.72/10.0 s, steps/err: 48(134.084939957 ms)/0.00121686123571)
branching: 28->40 (0.72/10.0 s, steps/err: 53(125.839948654 ms)/0.00199773071659)
try: 0 - (0, 0)->(1, 0)
try: 39 - (1, 1)->(2, 1)
transition motion tried: True
try transition motion
try: 22 - (0, 1)->(1, 1)
try: 7 - (0, 1)->(1, 1)
result: 30 - (0, 1)->(1, 1) = success
try: 40 - (1, 1)->(2, 1)
try constrained motion
transition motion tried: True
branching: 30->41 (0.75/10.0 s, steps/err: 48(129.194974899 ms)/0.00170333567639)
try transition motion
try transition motion
result: 4 - (0, 1)->(1, 1) = success
branching: 4->42 (0.76/10.0 s, steps/err: 48(116.232872009 ms)/0.0014702271153)
try constrained motion
try: 41 - (1, 1)->(2, 1)
try: 42 - (1, 1)->(2, 1)
try constrained motion
transition motion tried: True
try constrained

try transition motion
branching: 49->57 (1.32/10.0 s, steps/err: 53(159.387111664 ms)/0.00188681346015)
try: 57 - (1, 1)->(2, 1)
try constrained motion
try: 58 - (0, 1)->(1, 1)
try constrained motion
joint max
try transition motion
try: 56 - (0, 1)->(1, 1)
constrained motion tried: False
try transition motion
try transition motion
joint max
constrained motion tried: False
Motion Plan Failure
Motion Plan Failure
result: 21 - (1, 1)->(2, 1) = fail
result: 25 - (1, 1)->(2, 1) = fail
joint min
constrained motion tried: False
Motion Plan Failure
result: 20 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(1, 0)
try: 0 - (0, 0)->(1, 0)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
joint max
try transition motion
constrained motion tried: False
Motion Plan Failure
result: 32 - (1, 1)->(2, 1) = fail
try: 4 - (0, 1)->(1, 1)
transition motion tried: True
result: 56 - (0, 1)->(1, 1) = success
branching: 56->59 (1.47/10.0 s, steps/err: 41(127.372980118 ms)/0.00174940443171)
try: 59 -

result: 0 - (0, 0)->(1, 0) = fail
======================= terminated 27: first answer acquired from other agent ===============================
transition motion tried: False
transition motion tried: True
Motion Plan Failure
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired from other agent ===============================
result: 0 - (0, 0)->(1, 0) = fail
result: 67 - (0, 1)->(1, 1) = success
branching: 67->74 (1.8/10.0 s, steps/err: 48(106.419086456 ms)/0.00146277787519)
end
======================= terminated 33: first answer acquired from other agent ===============================
======================= terminated 29: first answer acquired from other agent ===============================
constrained motion tried: True
======================= terminated 35: first answer acquired from other agent ===============================
Goal reached
result: 9 - (1, 1)->(2, 

...

try transition motion
try: 366 - (0, 1)->(1, 1)
constrained motion tried: False
try transition motion
Motion Plan Failure
constrained motion tried: False
Motion Plan Failure
try transition motion
joint max
constrained motion tried: False
result: 338 - (1, 1)->(2, 1) = fail
result: 335 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
joint max
Motion Plan Failure
result: 345 - (1, 1)->(2, 1) = fail
constrained motion tried: False
Motion Plan Failure
try: 0 - (0, 0)->(1, 0)
result: 342 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try: 0 - (0, 0)->(0, 1)
try: 0 - (0, 0)->(0, 1)
joint max
try transition motion
try transition motion
constrained motion tried: False
try transition motion
try transition motion
try transition motion
Motion Plan Failure
result: 356 - (1, 1)->(2, 1) = fail
transition motion tried: True
transition motion tried: True
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
try: 164 - (0, 1)->(1, 1)
transition motion tried: True
result: 0 - (0, 0)->(0, 1

try transition motion
try: 266 - (0, 1)->(1, 1)
try: 4 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
try transition motion
result: 153 - (0, 1)->(1, 1) = success
branching: 153->384 (9.87/10.0 s, steps/err: 59(420.230150223 ms)/0.00200805010142)
transition motion tried: True
result: 375 - (0, 1)->(1, 1) = success
try: 384 - (1, 1)->(2, 1)
transition motion tried: True
result: 376 - (0, 1)->(1, 1) = success
branching: 375->385 (9.9/10.0 s, steps/err: 46(136.940002441 ms)/0.00192550582051)
transition motion tried: False
branching: 376->386 (9.91/10.0 s, steps/err: 46(134.235858917 ms)/0.00169331902978)
transition motion tried: True
Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
try: 385 - (1, 1)->(2, 1)
result: 377 - (0, 1)->(1, 1) = success
transition motion tried: True
branching: 377->387 (9.92/10.0 s, steps/err: 55(165.642976761 ms)/0.00161957038105)
try constrained motion
try: 387 - (1, 1)->(2, 1)
joint max
try constrained motion
joint max
try: 386 - (1,

======================= terminated 10: max iteration time reached (1633429196/1633429186.45 s) ===============================
transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
transition motion tried: True
branching: 4->398 (10.08/10.0 s, steps/err: 52(225.793838501 ms)/0.00119681261799)
transition motion tried: True
result: 280 - (0, 1)->(1, 1) = success
joint max
constrained motion tried: False
Motion Plan Failure
joint max
branching: 280->399 (10.09/10.0 s, steps/err: 48(119.437932968 ms)/0.001919876697)
======================= terminated 32: max iteration time reached (1633429196/1633429186.45 s) ===============================
transition motion tried: True
result: 380 - (0, 1)->(1, 1) = success
constrained motion tried: False
Motion Plan Failure
result: 371 - (1, 1)->(2, 1) = fail
result: 362 - (1, 1)->(2, 1) = fail
result: 372 - (0, 1)->(1, 1) = success
======================= terminated 11: max iteration time reached (1633429196/1633429186.45 s) ===============

Motion Plan Failure
result: 379 - (1, 1)->(2, 1) = fail
joint max
======================= terminated 6: max iteration time reached (1633429196/1633429186.45 s) ===============================
constrained motion tried: False
Motion Plan Failure
result: 390 - (1, 1)->(2, 1) = fail
======================= terminated 3: max iteration time reached (1633429196/1633429186.45 s) ===============================
collision
constrained motion tried: False
transition motion tried: False
Motion Plan Failure
Motion Plan Failure
result: 369 - (1, 1)->(2, 1) = fail
result: 0 - (0, 0)->(1, 0) = fail
======================= terminated 14: max iteration time reached (1633429196/1633429186.45 s) ===============================
======================= terminated 5: max iteration time reached (1633429196/1633429186.45 s) ===============================
joint max
constrained motion tried: False
Motion Plan Failure
result: 378 - (1, 1)->(2, 1) = fail
======================= terminated 2: max iteration time rea

IndexError: list index out of range

## TODO
* 실제 동작 생성 조건 추가하기

In [ ]:

sweep_width, (area_width, area_depth), width_range, divisions, div_num, area_center = select_task_area(
    robot_config, TABLE_DIMS, TOOL_DIM, EE_DEPTH_OFF, EE_HEIGHT, ROBOT_Z_ANGLE, 
    MARGIN=MARGIN, TOOL_DEPTH_MIN=0.6, TOOL_DEPTH_MAX=1.0, allow_back_approach=True)

In [ ]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

table_name = "bed_mat"

# floor
floor = pscene.create_binder(bname="floor", gname="floor_ws", _type=PlacePlane, point=None)

# waypoints
TABLE_DIM_DW = np.array(TABLE_DIMS[:2])
track_size_dw = TABLE_DIM_DW / (divisions[1], divisions[0])
REF_CENTER_00 = (TABLE_DIM_DW/2 - track_size_dw / 2) * (-1,1)

Trt = SE3(Rot_axis(3,np.pi), tuple(np.negative(area_center))+(TABLE_DIMS[2]-TRACK_THICKNESS/2,)) # offset from indy base to selected track
Tmr = get_tf(to_link=ROBOT_BASE, joint_dict=HOME_DICT, urdf_content=gscene.urdf_content, from_link=MOBILE_BASE) # from mobile to indy
Ttm = SE3_inv(np.matmul(Tmr, Trt)) # from track to mobile
Tfb = SE3_inv(gscene.NAME_DICT["floor_ws"].get_tf(HOME_DICT)) # from floor to base_link
Tfm_home = np.matmul(Tfb, get_tf(to_link=MOBILE_BASE, joint_dict=HOME_DICT, urdf_content=gscene.urdf_content)) # from floor to mobile home

T_tab_trac_list = []
track_list = []
wp_list = []
for i_d in range(divisions[1]):
    for i_w in range(divisions[0]):
        xyz = (tuple((REF_CENTER_00 + track_size_dw * (i_d, -i_w)) * ((1, -1) if i_d == 1 else (1, 1)))
               +(TABLE_DIMS[2]/2-TRACK_THICKNESS/2,))
        rpy = (0,0,np.pi) if i_d == 1 else (0,0,0)
        track_tmp = gscene.create_safe(GEOTYPE.BOX, "track_{}_{}".format(i_d, i_w), "base_link",
                                       dims = tuple(track_size_dw)+(TRACK_THICKNESS,), center = xyz, rpy = rpy, 
                                       color=(0.0,0.0,0.8,0.3), display=True, fixed=True, collision=False, 
                                       parent=table_name)
        track_list.append(track_tmp)
        Tft = np.matmul(Tfb, track_tmp.get_tf(HOME_DICT))
        Tfm = np.matmul(Tft, Ttm)
        xyz, rpy = T2xyzrpy(Tfm)
        wp_list.append(
            gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(len(wp_list)), link_name="base_link",
                               dims=WP_DIMS, center=tuple(xyz[:2])+(0,), 
                               rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, 
                               parent="floor_ws")
        )

#### 1.2.2 Generate motions to cleaning locations

#### 1.2.3 Generate cleaning sweep motions

### 1.3 Execute bed cleaning sequence


## 2. Closet cleaning

### 2.1. Detect closet

#### 2.1.1 Move to closet side of bed

#### 2.1.2 Detect the closet and add to scene

In [ ]:
closet_center=(3,-0.75,0), closet_rpy=(0,0,np.pi/2)

add_closet(COLOR_CLOSET_COL=(0,1,0,0.3))

### 2.2. Make closet cleaning plan

#### 2.2.1 Select cleaning location

#### 2.2.2 Generate cleaning sweep motions

## 3. Return to home